# Fine-Tune a Generative AI Model for Dialogue Summarization

### 1.1 - Set up Kernel and Required Dependencies

In [1]:
import os

%pip install torch torchdata  --index-url https://download.pytorch.org/whl/cu118 --quiet

%pip install transformers datasets evaluate rouge_score peft --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

torch.cuda.is_available()

C:\Users\suppo\MLENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### 1.2 - Load Dataset and LLM

In [26]:
DATASET = "DIBT/10k_prompts_ranked"

dataset = load_dataset(DATASET, split='train')

dataset = dataset.filter(lambda x: len(x["prompt"]) > 50 and len(x["prompt"]) <= 300, batched=False)

dataset = dataset.train_test_split(test_size=0.1)

dataset

Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10331/10331 [00:00<00:00, 35739.35 examples/s]


DatasetDict({
    train: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 4680
    })
    test: Dataset({
        features: ['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', 'raw_responses', 'kind', 'cluster_description', 'topic'],
        num_rows: 521
    })
})

In [27]:
dataset['train']['prompt'][4]

'describe how you would build a slim stack board-to-board connector'

Load the pre-trained [FLAN-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5) and its tokenizer directly from HuggingFace. Notice that you will be using the [small version](https://huggingface.co/google/flan-t5-base) of FLAN-T5. Setting `torch_dtype=torch.bfloat16` specifies the memory type to be used by this model.

In [28]:
model_name='google/flan-t5-small'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [29]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 76961152
all model parameters: 76961152
percentage of trainable model parameters: 100.00%


### 1.3 - Test the Model with Zero Shot Inferencing

In [33]:
index = 5

orginal_prompt = dataset['train'][index]['prompt']

def make_safe_prompt(prompt):
    return f"""Create a confidential prompt from the following prompt:

{prompt}

Prompt:"""

prompt = make_safe_prompt(orginal_prompt)

inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{orginal_prompt}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
How does the Federal Aviation Administration ensure air traffic control is safe and efficient?
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The Federal Aviation Administration has a mandate to ensure that air traffic control is safe and efficient.


## 2 - Perform Full Fine-Tuning

In [34]:
def tokenize_function(batch):
    prompts = [make_safe_prompt(pp) for pp in batch["prompt"]]
    batch['input_ids'] = tokenizer(prompts, padding="max_length", truncation=True, return_tensors="pt").to('cuda').input_ids
    batch['labels'] = tokenizer(batch["prompt"], padding="max_length", truncation=True, return_tensors="pt").to('cuda').input_ids
    return batch

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['prompt', 'quality', 'metadata', 'avg_rating', 'num_responses', 'agreement_ratio', \
                                                        'raw_responses', 'kind', 'cluster_description', 'topic'])

tokenized_datasets

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 521/521 [00:00<00:00, 2404.35 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 4680
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 521
    })
})

Check the shapes of all three parts of the dataset:

In [10]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (9297, 2)
Test: (1034, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 9297
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1034
    })
})


<a name='3'></a>
## 3 - Perform Parameter Efficient Fine-Tuning (PEFT)

Now, let's perform **Parameter Efficient Fine-Tuning (PEFT)** fine-tuning as opposed to "full fine-tuning" as you did above. PEFT is a form of instruction fine-tuning that is much more efficient than full fine-tuning - with comparable evaluation results as you will see soon. 

PEFT is a generic term that includes **Low-Rank Adaptation (LoRA)** and prompt tuning (which is NOT THE SAME as prompt engineering!). In most cases, when someone says PEFT, they typically mean LoRA. LoRA, at a very high level, allows the user to fine-tune their model using fewer compute resources (in some cases, a single GPU). After fine-tuning for a specific task, use case, or tenant with LoRA, the result is that the original LLM remains unchanged and a newly-trained “LoRA adapter” emerges. This LoRA adapter is much, much smaller than the original LLM - on the order of a single-digit % of the original LLM size (MBs vs GBs).  

That said, at inference time, the LoRA adapter needs to be reunited and combined with its original LLM to serve the inference request.  The benefit, however, is that many LoRA adapters can re-use the original LLM which reduces overall memory requirements when serving multiple tasks and use cases.

<a name='3.1'></a>
### 3.1 - Setup the PEFT/LoRA model for Fine-Tuning

You need to set up the PEFT/LoRA model for fine-tuning with a new layer/parameter adapter. Using PEFT/LoRA, you are freezing the underlying LLM and only training the adapter. Have a look at the LoRA configuration below. Note the rank (`r`) hyper-parameter, which defines the rank/dimension of the adapter to be trained.

In [11]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

Add LoRA adapter layers/parameters to the original LLM to be trained.

In [12]:
peft_model = get_peft_model(original_model, lora_config).to('cuda')
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 1376256
all model parameters: 78337408
percentage of trainable model parameters: 1.76%


<a name='3.2'></a>
### 3.2 - Train PEFT Adapter

Define training arguments and create `Trainer` instance.

In [13]:
output_dir = f'./peft-train-checkpoint'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    use_cpu=False,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

In [14]:
peft_trainer.train()

peft_model_path=f"./model_checkpoint"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
500,3.136100
1000,1.629300


('./model_checkpoint\\tokenizer_config.json',
 './model_checkpoint\\special_tokens_map.json',
 './model_checkpoint\\tokenizer.json')

In [15]:
from peft import PeftModel, PeftConfig


original_model1 = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to('cuda')

peft_model1 = PeftModel.from_pretrained(original_model1, 
                                       peft_model_path, 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

print(print_number_of_trainable_model_parameters(peft_model1))

trainable model parameters: 0
all model parameters: 78337408
percentage of trainable model parameters: 0.00%


The number of trainable parameters will be `0` due to `is_trainable=False` setting:

<a name='3.3'></a>
### 3.3 - Evaluate the Model Qualitatively (Human Evaluation)

Make inferences for the same example as in sections [1.3](#1.3) and [2.3](#2.3), with the original model, fully fine-tuned and PEFT model.

In [16]:
index = 26

original_prompt = dataset['test'][index]['prompt']

prompt = make_safe_prompt(original_prompt)

input_ids = tokenizer(prompt, return_tensors="pt").to('cuda').input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)


peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(f'ORIGINAL PROMPT:\n{original_prompt}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'PEFT MODEL:\n{peft_model_text_output}')

ORIGINAL PROMPT:
We will be working on Justin McDonald's profile as a copywriter and consultant. The goal in this chat is to really make him shine and land high-paying clients for private consulting, and freelance copywriting gigs that pay well and are fun to do.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
We will be working on Justin McDonald's profile as a copywriter and consultant. The goal in this chat is to really make him shine and land high-paying clients for private consulting, and freelance copywriting gigs that pay well and are fun to do.
---------------------------------------------------------------------------------------------------
PEFT MODEL:
We will be working on Justin McDonald's profile as a copywriter and consultant. The goal in this chat is to really make him shine and land high-paying clients for private consulting, and freelance copywriting gigs that pay well and are fun to do. Prompt: We wil

<a name='3.4'></a>
### 3.4 - Evaluate the Model Quantitatively (with ROUGE Metric)
Perform inferences for the sample of the test dataset (only 10 dialogues and summaries to save time). 

In [17]:
original_prompts = []
original_model_prompts = []
peft_model_prompts = []


for safe_prompt, original_prompt in zip(tokenized_datasets['test'][0:10]['input_ids'], tokenized_datasets['test'][0:10]['labels']):
    original_prompts.append( tokenizer.decode(original_prompt, skip_special_tokens=True) )

    inp = torch.unsqueeze(torch.as_tensor(safe_prompt), 0).to('cuda')
    
    output = original_model.generate(input_ids=inp, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_prompts.append( tokenizer.decode(output[0], skip_special_tokens=True) )

    output = peft_model.generate(input_ids=inp, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_prompts.append( tokenizer.decode(output[0], skip_special_tokens=True) )

zipped = list(zip(peft_model_prompts, original_model_prompts, peft_model_prompts))
 
df = pd.DataFrame(zipped, columns = ['original_prompts', 'original_model_prompts', 'peft_model_prompts'])
df

,original_prompts,original_model_prompts,peft_model_prompts
0,You are given a code snippet that handles the ...,You are given a code snippet that handles the ...,You are given a code snippet that handles the ...
1,Transform the following sentence into an inter...,Transform the following sentence into an inter...,Transform the following sentence into an inter...
2,"How can I adapt ShareGPT, an open source chrom...","How can I adapt ShareGPT, an open source chrom...","How can I adapt ShareGPT, an open source chrom..."
3,As the project lead of a machine learning star...,As the project lead of a machine learning star...,As the project lead of a machine learning star...
4,"For a $1,000 investment, what is the future va...","For a $1,000 investment, what is the future va...","For a $1,000 investment, what is the future va..."
5,Can we ensure clear communication and customer...,How can we handle a dissatisfied customer whos...,Can we ensure clear communication and customer...
6,i am implementing pagination for our chat for ...,i am implementing pagination for our chat for ...,i am implementing pagination for our chat for ...
7,I want you to act as a math teacher. I will pr...,I want you to act as a math teacher. I will pr...,I want you to act as a math teacher. I will pr...
8,"There is this weird youtuber, who does straigh...","There is this weird youtuber, who does straigh...","There is this weird youtuber, who does straigh..."
9,asin(0.5) / acos(0.5) - tan(0.5)= Prompt,asin(0.5) / acos(0.5) - tan(0.5)=,asin(0.5) / acos(0.5) - tan(0.5)= Prompt


You already computed ROUGE score on the full dataset, after loading the results from the `data/dialogue-summary-training-results.csv` file. Load the values for the PEFT model now and check its performance compared to other models.

In [18]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=df['original_model_prompts'],
    references=df['original_prompts'],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=df['peft_model_prompts'],
    references=df['original_prompts'],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.9144940307047925, 'rouge2': 0.9000550693522074, 'rougeL': 0.9143515620826237, 'rougeLsum': 0.9147603702296172}
PEFT MODEL:
{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


The results show less of an improvement over full fine-tuning, but the benefits of PEFT typically outweigh the slightly-lower performance metrics.

Calculate the improvement of PEFT over the original model:

In [19]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 8.55%
rouge2: 9.99%
rougeL: 8.56%
rougeLsum: 8.52%
